#### importing modules & reproducibility

In [26]:
# import modules
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt

# Fixing random state for reproducibility
np.random.seed(387)

# ***Preprocessing***

In [27]:
# read data
characters = pd.read_csv('datasets/Characters.csv', sep=';')
hp1 = pd.read_csv('datasets/Harry Potter 1.csv', sep=';')
hp2 = pd.read_csv('datasets/Harry Potter 2.csv', sep=';')
hp3 = pd.read_csv('datasets/Harry Potter 3.csv', sep=';')

## ***profiling structure***
### fixing books data

In [28]:
trilogy = [hp1, hp2, hp3] 

# changing books columns and values to lower case
'''if not other cols would be created because of name differences'''
for book in trilogy:
    book.columns = hp1.columns.str.lower()
    book['character'] = book['character'].str.lower() # character column
    book['sentence'] = book['sentence'].str.lower()   # sentence column

In [29]:
# vertical concatenation of books
combined = pd.concat([hp1, hp2, hp3], ignore_index=True)
combined.head()

,character,sentence
0,dumbledore,"i should've known that you would be here, prof..."
1,mcgonagall,"good evening, professor dumbledore."
2,mcgonagall,"are the rumors true, albus?"
3,dumbledore,"i'm afraid so, professor."
4,dumbledore,the good and the bad.


### fixing characters data

In [30]:
# changing books columns to lower case
'''if not other cols would be created because of name differences'''
characters.columns = characters.columns.str.lower()

In [47]:
# columns to date format
characters['birth'] = pd.to_datetime(characters['birth'], errors='coerce')
characters['death'] = pd.to_datetime(characters['death'], errors='coerce')

In [89]:
# get non numerical values
non_num = characters.select_dtypes(exclude=['int64','float64','datetime','timedelta'])
num     = characters.select_dtypes(exclude=['object'])

In [101]:
#
non_num.columns

Index(['name', 'gender', 'job', 'house', 'wand', 'patronus', 'species',
       'blood status', 'hair colour', 'eye colour', 'loyalty', 'skills'],
      dtype='object')

In [146]:
'''---might change it in a future'''
# changing book values to lowercase

non_num['name'] = non_num['name'].str.lower()
non_num['gender']	= non_num['gender'].str.lower()
non_num['job']	= non_num['job'].str.lower()	
non_num['house'] = non_num['house'].str.lower()	
non_num['wand']	= non_num['wand'].str.lower()
non_num['patronus']	= non_num['patronus'].str.lower()
non_num['species'] = non_num['species'].str.lower()
non_num['blood status']	= non_num['blood status'].str.lower()	
non_num['hair colour'] = non_num['hair colour'].str.lower()
non_num['eye colour'] =	non_num['eye colour'].str.lower()
non_num['loyalty'] = non_num['loyalty']	.str.lower()
non_num['skills']  = non_num['skills'].str.lower()
non_num

,name,gender,job,house,wand,patronus,species,blood status,hair colour,eye colour,loyalty,skills
0,harry james potter,male,student,gryffindor,"11"" holly phoenix feather",stag,human,half-blood,black,bright green,albus dumbledore | dumbledore's army | order o...,parseltongue| defence against the dark arts | ...
1,ronald bilius weasley,male,student,gryffindor,"12"" ash unicorn tail hair",jack russell terrier,human,pure-blood,red,blue,dumbledore's army | order of the phoenix | hog...,wizard chess | quidditch goalkeeping
2,hermione jean granger,female,student,gryffindor,"10¾"" vine wood dragon heartstring",otter,human,muggle-born,brown,brown,dumbledore's army | order of the phoenix | hog...,almost everything
3,albus percival wulfric brian dumbledore,male,headmaster,gryffindor,"15"" elder thestral tail hair core",phoenix,human,half-blood,silver| formerly auburn,blue,dumbledore's army | order of the phoenix | hog...,considered by many to be one of the most power...
4,rubeus hagrid,male,keeper of keys and grounds | professor of care...,gryffindor,"16"" oak unknown core",none,half-human/half-giant,part-human (half-giant),black,black,albus dumbledore | order of the phoenix | hogw...,resistant to stunning spells| above average st...
...,...,...,...,...,...,...,...,...,...,...,...,...
135,wilhelmina grubbly-plank,female,substitute professor of care of magical creatu...,NaN,unknown,non-corporeal,human,NaN,grey,NaN,hogwarts school of witchcraft and wizardry,NaN
136,fenrir greyback,male,NaN,NaN,unknown,none,werewolf,NaN,grey,NaN,lord voldemort | death eaters,physical combat
137,gellert grindelwald,male,revolutionary leader(c. 1920s[6] – 1945),NaN,"15"", elder, thestral tail hair core",NaN,human,pure-blood or half-blood,blond,blue,gellert grindelwald's acolytes,duelling
138,dobby,male,"malfoy family's house-elf (? - 1993),\nhogwart...",NaN,NaN,NaN,house elf,NaN,green,NaN,NaN,"a type of magic specific to house-elves, perfo..."


#### inspecting tables

In [11]:
# column names
print(characters.columns,'\n')
print(hp1.columns)

Index(['id', 'name', 'gender', 'job', 'house', 'wand', 'patronus', 'species',
       'blood status', 'hair colour', 'eye colour', 'loyalty', 'skills',
       'birth', 'death'],
      dtype='object') 

Index(['character', 'sentence'], dtype='object')


# Plots :
### Characters with the most sentences
#### top 15 of the first 3 movies 